In [1]:
%load_ext sql
%sql sqlite:///Data.db

'Connected: @Data.db'

## Question 1
Which brand saw the most dollars spent in the month of June?

In [2]:
%%sql
SELECT  b.NAME, ROUND(sum(i.TOTAL_FINAL_PRICE),2) as "Total Sales in June"
FROM Brands b,  Items i
Where i.MODIFY_DATE LIKE '%-06-%'
AND (b.BRAND_CODE = i.BRAND_CODE
OR b.BARCODE = i.BARCODE)
GROUP BY b.NAME
ORDER BY sum(i.TOTAL_FINAL_PRICE) DESC
LIMIT 1;

 * sqlite:///Data.db
Done.


NAME,Total Sales in June
Keebler,179943.7


## Question 2
Which user spent the most money in the month of August?

In [3]:
%%sql
SELECT u.ID as "USER ID", ROUND(sum(i.TOTAL_FINAL_PRICE),2) as "Total Spends in August"
FROM Items i, Receipts r, User u
WHERE i.MODIFY_DATE LIKE '%-08-%'
AND i.REWARDS_RECEIPT_ID = r.ID
AND r.USER_ID = u.ID
GROUP BY u.ID
ORDER BY sum(i.TOTAL_FINAL_PRICE) DESC
LIMIT 1;

 * sqlite:///Data.db
Done.


USER ID,Total Spends in August
609ab37f7a2e8f2f95ae968f,157719.27


## Question 3
What user bought the most expensive item?

There my be the case there are two user purchased the same expensive item. The fllowing code output one user

In [12]:
%%sql
SELECT u.ID as "User that purchased the most expensive item", u.STATE as "State", e.TOTAL_FINAL_PRICE as "Price of the most expensive item"
FROM Receipts r, User u, (SELECT * 
FROM Items
ORDER BY TOTAL_FINAL_PRICE DESC
LIMIT 1) as e
Where e.REWARDS_RECEIPT_ID = r.ID
AND r.USER_ID = u.ID;


 * sqlite:///Data.db
Done.


User that purchased the most expensive item,State,Price of the most expensive item
617376b8a9619d488190e0b6,NY,31005.99


To output all the users that purchased the most expensive item

In [11]:
%%sql
SELECT u.ID as "User that purchased the most expensive item", u.STATE as "State", e.TOTAL_FINAL_PRICE as "Price of the most expensive item"
FROM Receipts r, User u, (SELECT * 
FROM Items
ORDER BY TOTAL_FINAL_PRICE DESC
LIMIT 1) as e, Items i
Where i.TOTAL_FINAL_PRICE = e.TOTAL_FINAL_PRICE
AND i.REWARDS_RECEIPT_ID = r.ID
AND r.USER_ID = u.ID;

 * sqlite:///Data.db
Done.


User that purchased the most expensive item,State,Price of the most expensive item
617376b8a9619d488190e0b6,NY,31005.99
617376b8a9619d488190e0b6,NY,31005.99
